BU PROJEDE YOLLARIN RİSK ORANINI TAHMİN ETMEYE ÇALIŞACAĞIZ.

In [1]:
import pandas as pd
import warnings
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
import numpy as np

warnings.filterwarnings('ignore')

In [2]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

In [3]:
df_train.head()

,id,road_type,num_lanes,curvature,speed_limit,lighting,weather,road_signs_present,public_road,time_of_day,holiday,school_season,num_reported_accidents,accident_risk
0,0,urban,2,0.06,35,daylight,rainy,False,True,afternoon,False,True,1,0.13
1,1,urban,4,0.99,35,daylight,clear,True,False,evening,True,True,0,0.35
2,2,rural,4,0.63,70,dim,clear,False,True,morning,True,False,2,0.30
3,3,highway,4,0.07,35,dim,rainy,True,True,morning,False,False,1,0.21
4,4,rural,1,0.58,60,daylight,foggy,False,False,evening,True,False,1,0.56


In [4]:
df_test.head()

,id,road_type,num_lanes,curvature,speed_limit,lighting,weather,road_signs_present,public_road,time_of_day,holiday,school_season,num_reported_accidents
0,517754,highway,2,0.34,45,night,clear,True,True,afternoon,True,True,1
1,517755,urban,3,0.04,45,dim,foggy,True,False,afternoon,True,False,0
2,517756,urban,2,0.59,35,dim,clear,True,False,afternoon,True,True,1
3,517757,rural,4,0.95,35,daylight,rainy,False,False,afternoon,False,False,2
4,517758,highway,2,0.86,35,daylight,clear,True,False,evening,False,True,3


In [5]:
sözlük = {True:1,
         False:0}

In [6]:
df_train['road_signs_present'] = df_train['road_signs_present'].map(sözlük)
df_train['public_road'] = df_train['public_road'].map(sözlük)
df_train['holiday'] = df_train['holiday'].map(sözlük)
df_train['school_season'] = df_train['school_season'].map(sözlük)

In [7]:
df_train.head()

,id,road_type,num_lanes,curvature,speed_limit,lighting,weather,road_signs_present,public_road,time_of_day,holiday,school_season,num_reported_accidents,accident_risk
0,0,urban,2,0.06,35,daylight,rainy,0,1,afternoon,0,1,1,0.13
1,1,urban,4,0.99,35,daylight,clear,1,0,evening,1,1,0,0.35
2,2,rural,4,0.63,70,dim,clear,0,1,morning,1,0,2,0.30
3,3,highway,4,0.07,35,dim,rainy,1,1,morning,0,0,1,0.21
4,4,rural,1,0.58,60,daylight,foggy,0,0,evening,1,0,1,0.56


In [8]:
df_train['speed_limit'].value_counts()

45    115752
35    111628
25    101422
60    100304
70     88648
Name: speed_limit, dtype: int64

In [9]:
#Yeni sütunda hız sınırı 50'den büyük olanlara 1 küçük olanlara 0 yazdık
df_train['Speed_limit_flag'] = (df_train['speed_limit']>50).astype(int)

In [10]:
df_train['curvature'].describe()

count    517754.000000
mean          0.488719
std           0.272563
min           0.000000
25%           0.260000
50%           0.510000
75%           0.710000
max           1.000000
Name: curvature, dtype: float64

In [11]:
#Eğimi 0.50 den büyük olanlara 1 küçük olanlara 0 yazdık.
df_train['curvature_limit_flag'] = (df_train['curvature']>0.5).astype(int)

In [12]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517754 entries, 0 to 517753
Data columns (total 16 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   id                      517754 non-null  int64  
 1   road_type               517754 non-null  object 
 2   num_lanes               517754 non-null  int64  
 3   curvature               517754 non-null  float64
 4   speed_limit             517754 non-null  int64  
 5   lighting                517754 non-null  object 
 6   weather                 517754 non-null  object 
 7   road_signs_present      517754 non-null  int64  
 8   public_road             517754 non-null  int64  
 9   time_of_day             517754 non-null  object 
 10  holiday                 517754 non-null  int64  
 11  school_season           517754 non-null  int64  
 12  num_reported_accidents  517754 non-null  int64  
 13  accident_risk           517754 non-null  float64
 14  Speed_limit_flag    

In [13]:
df_train.corr()['accident_risk'].sort_values(ascending=False)

accident_risk             1.000000
curvature                 0.543946
curvature_limit_flag      0.516083
Speed_limit_flag          0.487876
speed_limit               0.430898
num_reported_accidents    0.213891
holiday                   0.051129
public_road               0.031032
id                        0.000969
road_signs_present        0.000629
school_season            -0.000977
num_lanes                -0.006003
Name: accident_risk, dtype: float64

In [14]:
df_train['num_reported_accidents'] = df_train['num_reported_accidents']**3

In [15]:
df_train['accident_num_lanes'] = df_train['num_reported_accidents'] * df_train['num_lanes']
df_train['curvature_speed'] = df_train['curvature'] * df_train['speed_limit']
df_train['curvature_accidents'] = df_train['curvature'] * df_train['num_reported_accidents']

In [16]:
df_train.corr()['accident_risk'].sort_values(ascending=False)

accident_risk             1.000000
curvature_speed           0.675621
curvature                 0.543946
curvature_limit_flag      0.516083
Speed_limit_flag          0.487876
speed_limit               0.430898
curvature_accidents       0.395320
num_reported_accidents    0.309962
accident_num_lanes        0.275388
holiday                   0.051129
public_road               0.031032
id                        0.000969
road_signs_present        0.000629
school_season            -0.000977
num_lanes                -0.006003
Name: accident_risk, dtype: float64

In [17]:
x2 = df_train.drop(['id','accident_risk'],axis=1)
y = df_train[['accident_risk']]

In [18]:
x1 =pd.get_dummies(x2)

In [19]:
from xgboost import XGBRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,r2_score,mean_absolute_error

In [20]:
x=MinMaxScaler().fit_transform(x1)

In [21]:
x_train, x_test, y_train, y_test=train_test_split(x,y,test_size=.10,random_state=42)

In [22]:
#BURADA BAŞARI ORANIMIZI GÖRMEK İÇİ SADECE TRAİN VERİ SETİ ÜZERİNDE ÇALIŞIP RMSE SKORUNU KONTROL EDİYORUZ.
xgb=XGBRegressor()
model = xgb.fit(x_train,y_train)
tahmin = model.predict(x_test)
mean_squared_error(y_test,tahmin)**.5

0.05614278965832049

In [23]:
#TEST VERİSİNİ UYUGUN HALE GETİRİYORUZ.
df_test['Speed_limit_flag'] = (df_test['speed_limit']>50).astype(int)
df_test['curvature_limit_flag'] = (df_test['curvature']>0.5).astype(int)
df_test['num_reported_accidents'] = df_test['num_reported_accidents']**3
df_test['accident_num_lanes'] = df_test['num_reported_accidents'] * df_test['num_lanes']
df_test['curvature_speed'] = df_test['curvature'] * df_test['speed_limit']
df_test['curvature_accidents'] = df_test['curvature'] * df_test['num_reported_accidents']
test_x = df_test.drop(['id'],axis=1)
test_x2 = pd.get_dummies(test_x)
test_x3=MinMaxScaler().fit_transform(test_x2)


In [24]:
#YUKARIDA TRAİN VERİ SETİ İLE YAPTIĞIMIZ TESTTE 0.056 RMSE SKOR ELDE ETTİK. BU İYİ BİR SONUÇ BU YÜZDEN MODELİ TRAİN TEST SPLİT İLE AYIRMADAN
#SADECE DF_TRAİN İLE EĞİTİP DF_TEST İLE TAHMİN SONUÇLARINI YAZDIRACAĞIZ. SONUÇLARI KAGGLE A YÜKLEYİM BAŞARI ORANIMIZI ORADAN GÖRECEĞİZ.
#BURADA MODELİ X VE Y İLE EĞİTİYORUZ. BU X_TRAİN VE Y_TRAİN YAPILMAMIŞ HALİ.
xgb=XGBRegressor()
kaggle_model = xgb.fit(x,y)
tahmin=kaggle_model.predict(test_x3)

In [25]:
#SONUÇLARI BOŞ BİR DATA FRAMEYE YAZDIRIM CSV OLARAK KAYDEDECEĞİZ.
new_data = pd.DataFrame()

In [26]:
new_data.index = df_test['id']

In [27]:
new_data['accident_risk'] = tahmin

In [28]:
new_data.head()

,accident_risk
id,
517754,0.290477
517755,0.122612
517756,0.171400
517757,0.307942
517758,0.409922


In [30]:
new_data.to_csv('PlayGroundPredict.csv')

In [31]:
#MODELİMİZİ KAYDEDİYORUZ.
import pickle

# Modeli kaydetme
with open("road_model.pkl", "wb") as file:
    pickle.dump(kaggle_model, file)